In [1]:
!pip install pandarallel

  Preparing metadata (setup.py) ... - \ done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=923c67d5cf2f6b8f864e4047079eafc41efcd3cd076972011de034274f7a4f0e
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime as dts
from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True) # initialize pandarallel

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
tqdm.pandas()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/processed/processed.pkl
/kaggle/input/df-mod2/df_mod2.pkl


In [3]:
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

In [4]:
def data_preparation(df):
    import pandas as pd
    #import numpy as np
    #from datetime import datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True,)
    df['step']=df['step'].astype(float)
    #print('.',end="")
    return(df)


In [5]:
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")

In [6]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  14508 non-null  object 
 1   night      14508 non-null  int64  
 2   event      14508 non-null  object 
 3   step       9585 non-null   float64
 4   timestamp  9585 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 566.8+ KB


In [7]:
df_events['timestamp']=df_events['timestamp'].parallel_apply(lambda x: pd.to_datetime(x, utc=True))
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   series_id  14508 non-null  object             
 1   night      14508 non-null  int64              
 2   event      14508 non-null  object             
 3   step       9585 non-null   float64            
 4   timestamp  9585 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(2)
memory usage: 566.8+ KB


In [8]:
mod_series=df_series.iloc[:20000000,:]

In [9]:
mod_series['timestamp']=mod_series['timestamp'].parallel_apply(lambda x: pd.to_datetime(x, utc=True))


/tmp/ipykernel_19/1195527709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_series['timestamp']=mod_series['timestamp'].parallel_apply(lambda x: pd.to_datetime(x, utc=True))


In [10]:
mod_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 5 columns):
 #   Column     Dtype              
---  ------     -----              
 0   series_id  object             
 1   step       uint32             
 2   timestamp  datetime64[ns, UTC]
 3   anglez     float32            
 4   enmo       float32            
dtypes: datetime64[ns, UTC](1), float32(2), object(1), uint32(1)
memory usage: 534.1+ MB


In [11]:
mod_series.to_pickle('df_mod2.pkl')

In [12]:
events=df_events[['series_id', 'step','event']]

In [13]:
df=pd.merge(mod_series,events,on=["step","series_id"],how='left')

In [14]:
df['sleep']=np.nan
df.loc[df["event"]=="onset", "sleep"] = 1
df.loc[df["event"]=="wakeup", "sleep"] = 0
df['sleep'].fillna(method='ffill', inplace=True)
df['sleep'].fillna(value=0.0, inplace=True)
df['sleep'].mean()

0.2377374

In [15]:
df.to_pickle('processed.pkl')

In [16]:
df=pd.read_pickle('/kaggle/input/processed/processed.pkl')

In [17]:
df

,series_id,step,timestamp,anglez,enmo,event,sleep
0,038441c925bb,0,2018-08-14 19:30:00+00:00,2.636700,0.0217,NaN,0.0
1,038441c925bb,1,2018-08-14 19:30:05+00:00,2.636800,0.0215,NaN,0.0
2,038441c925bb,2,2018-08-14 19:30:10+00:00,2.637000,0.0216,NaN,0.0
3,038441c925bb,3,2018-08-14 19:30:15+00:00,2.636800,0.0213,NaN,0.0
4,038441c925bb,4,2018-08-14 19:30:20+00:00,2.636800,0.0215,NaN,0.0
...,...,...,...,...,...,...,...
19999995,25e2b3dd9c3b,598695,2019-01-18 07:01:15+00:00,-35.585098,0.0011,NaN,0.0
19999996,25e2b3dd9c3b,598696,2019-01-18 07:01:20+00:00,-35.618500,0.0011,NaN,0.0
19999997,25e2b3dd9c3b,598697,2019-01-18 07:01:25+00:00,-35.635601,0.0010,NaN,0.0
19999998,25e2b3dd9c3b,598698,2019-01-18 07:01:30+00:00,-35.669399,0.0009,NaN,0.0


In [18]:
train_df=df[['timestamp','step','anglez','enmo','sleep']]

In [19]:
train_df

,timestamp,step,anglez,enmo,sleep
0,2018-08-14 19:30:00+00:00,0,2.636700,0.0217,0.0
1,2018-08-14 19:30:05+00:00,1,2.636800,0.0215,0.0
2,2018-08-14 19:30:10+00:00,2,2.637000,0.0216,0.0
3,2018-08-14 19:30:15+00:00,3,2.636800,0.0213,0.0
4,2018-08-14 19:30:20+00:00,4,2.636800,0.0215,0.0
...,...,...,...,...,...
19999995,2019-01-18 07:01:15+00:00,598695,-35.585098,0.0011,0.0
19999996,2019-01-18 07:01:20+00:00,598696,-35.618500,0.0011,0.0
19999997,2019-01-18 07:01:25+00:00,598697,-35.635601,0.0010,0.0
19999998,2019-01-18 07:01:30+00:00,598698,-35.669399,0.0009,0.0


In [20]:
import plotly.express as px
figure= px.imshow(train_df.corr(), text_auto=True, width=1200, height=1200)
figure.show()

## Not significant correlation between the columns
- Let's add another column 'Cluster' based on anglez and enmo
- Let's keep the number of clusters hoping we can achive some correlation

In [21]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler    

X=df[['anglez','enmo']]
#Scalling the data
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
#perform clustering
model=KMeans(n_clusters=5)
model.fit(X_scaled)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=5)

In [22]:
train_df['cluster']=model.labels_

/tmp/ipykernel_19/2373440302.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
figure= px.imshow(train_df.corr(), text_auto=True, width=1200, height=1200)
figure.show()